In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression


In [2]:
df = pd.read_csv("Ofertas_finales.csv",sep="|")

In [73]:
df.columns

Index(['ID_Subtipo', 'ID_Estrato', 'Canon', 'Area_Construida', 'Area_Privada',
       'Antiguedad', 'Piso', 'Habitaciones', 'Banios', 'Parqueadero',
       'Incluye_admin'],
      dtype='object')

In [3]:
#### Preproccesing
def DroppingVariables(var: list, df):
    return df.drop(var,axis=1)


In [6]:
df = DroppingVariables(['ciudad','Longitud','Latitud',
                'Administración','Descripcion'],df)

In [7]:
%pwd

'd:\\PABLO\\Streamapp'

In [87]:
from tqdm import tqdm

In [88]:
df_completo = pd.DataFrame()
for kan in tqdm(df['ID_Estrato'].unique()):
    for kun in df['ID_Subtipo'].unique():
        paso = df.loc[(df['ID_Estrato']==kan)&(df['ID_Subtipo']==kun)]
        std = np.std(paso['Canon'])
        mean = np.mean(paso['Canon'])
        df_completo = pd.concat([df_completo
                    ,paso.loc[(paso['Canon']>mean-2*std)&(paso['Canon']<mean+2*std)]])


100%|██████████| 8/8 [00:00<00:00, 23.12it/s]


In [90]:
df = df_completo.copy()

In [91]:
X = df.drop(['Canon'],axis=1)
y= df[['Canon','ID_Subtipo']]

In [92]:
col_categoricos = X.select_dtypes("object").columns
col_categoricos = col_categoricos.drop('ID_Subtipo')
col_numericos = X.select_dtypes(np.number).columns

In [93]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [95]:
#['estrato_0',
#'estrato_1','estrato_2','estrato_3','estrato_4','estrato_5','estrato_6','estrato_7','Admin_NO','Admin_Si']

In [96]:
for i in X['ID_Subtipo'].unique():
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder()
    preprocessor = ColumnTransformer(
        transformers=[
            ("categorical",categorical_transformer,col_categoricos),
                ("numerico",numeric_transformer, col_numericos)
            ]
    )
    clf = Pipeline(
        steps=[('preprocessor',preprocessor)]
    )

    clf_modelo = Pipeline(
        steps=[("Preprocesando",clf),("Modelando",GradientBoostingRegressor(random_state=19))]
    )

    #### 
    from sklearn.model_selection import train_test_split
    import joblib
    X_train,X_test,y_train,y_test = train_test_split(X.query(f"ID_Subtipo=='{i}'")
                        ,y.query(f"ID_Subtipo=='{i}'")['Canon'],test_size=0.3)

    #### to export my preproccesor
    clf.fit(X_train)
    #joblib.dump(clf, "transformador.pkl")

    ###
    clf_modelo.fit(X_train,y_train)
    joblib.dump(clf_modelo,f"Modelo_{i}.pkl")

In [60]:
mode = joblib.load(f"Modelo_{i}.pkl")

In [65]:
X_predict = pd.DataFrame(columns = ['ID_Estrato', 'Area_Construida', 'Area_Privada',
       'Antiguedad', 'Piso', 'Habitaciones', 'Banios', 'Parqueadero',
       'Incluye_admin'])

In [71]:
X_predict.loc[0] = ['1',34.37,40.45,20,3,1,1,1,'NO']

In [72]:
mode.predict(X_predict)

array([5416485.91679876])